Importing the required libraries is the first thing to do. I use numpy and panda for data manupilation, and sklearn for implementing models.

In [1]:
import numpy as np
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression


Using read_csv I can access the csv file extracted from RapidMiner.
I then checked to make sure that no None value field is remaining.

In [3]:
df = read_csv (f"F:\\Courses\\Data Science bootcamp\\archive\\telco_customer_pre.csv")

In [4]:
df.isnull().sum()

gender              0
Partner             0
Dependents          0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Churn               0
customerID          0
SeniorCitizen       0
tenure              0
MonthlyCharges      0
TotalCharges        0
dtype: int64

Here I dropped the customerID, as it should not impact the models.

In [5]:
ndf = df.drop(["customerID"], axis = 1)

In [6]:
ndf.head()

,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,29.85,29.85
1,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0,34,56.95,1889.50
2,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,2,53.85,108.15
3,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.0,0.0,0,45,42.30,1840.75
4,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,2,70.70,151.65


In [7]:
ndf.shape

(7043, 20)

I then separated the data from lable. Label in this case was Churn. Dataset is then split between test and train.

In [8]:
X = ndf.iloc[:, :-1].values
y = ndf.iloc[:, 4].values 

# The first model to use is K-Nearest Neighbors

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20) 


Here I standardized features by removing mean and scaling to unit variance


In [10]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) 


I train the KNN classifier and use it for prediction. The results are visible below

In [11]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train) 


KNeighborsClassifier()

In [12]:
y_predict = classifier.predict(X_test)

print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict)) 

[[126   0   0]
 [  0 653  29]
 [  0  12 589]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       126
         1.0       0.98      0.96      0.97       682
         2.0       0.95      0.98      0.97       601

    accuracy                           0.97      1409
   macro avg       0.98      0.98      0.98      1409
weighted avg       0.97      0.97      0.97      1409



# The Second model to check is Naive Bayes:
I first organize the data and then define the model.

In [13]:
labels = y
features = X
print('Class label = ', labels[0])
print(features[0])

Class label =  0.0
[ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    1.   29.85]


In [15]:
# Split dataset into random train and test subsets:
train, test, train_labels, test_labels = train_test_split(features, labels, test_size=0.33, random_state=42)

# Initialize classifier:
gnb = GaussianNB()

# Train the classifier:
model = gnb.fit(train, train_labels)

# Make predictions with the classifier:
predictive_labels = gnb.predict(test)
print(predictive_labels)

# Evaluate label (subsets) accuracy:
print(accuracy_score(test_labels, predictive_labels))

[0. 2. 1. ... 2. 2. 2.]
1.0


# The Last model to check is Binary Regression:

Similar steps were taken here.

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [17]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 1.00


In [18]:
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[ 205    0    0]
 [   0 1016    0]
 [   0    0  892]]


In [19]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       205
         1.0       1.00      1.00      1.00      1016
         2.0       1.00      1.00      1.00       892

    accuracy                           1.00      2113
   macro avg       1.00      1.00      1.00      2113
weighted avg       1.00      1.00      1.00      2113

